In [1]:
# Instalar o colab-xterm para acesso ao terminal via notebook
!pip install -q colab-xterm
%load_ext colabxterm

In [7]:
# Instalar ollama com
# curl -fsSL https://ollama.com/install.sh | sh
# ollama pull llama2
%xterm

Launching Xterm...

In [2]:
# Demais dependências para o processamento
# de linguagem natural, gerenciamento de
# embeddings e scraping de dados.
!pip install -q langchain chromadb bs4
!pip install -q langchain-community
!pip install -U langchain-ollama

In [3]:
# Dependências para arquivo no mesmo folder
!pip install -q langchain PyMuPDF

In [4]:
filename = "ISTOB - MAterial para certificação.pdf"

In [5]:
# Abre arquivo
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(filename)
documents = loader.load()

In [6]:
# Divide o conteúdo em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

print(f'Arquivo dividido em {len(chunks)} chunks')

Arquivo dividido em 273 chunks


In [8]:
!pip install -q tqdm

In [9]:
# Criação de embeddings
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from tqdm import tqdm

# Inicialização
embeddings = OllamaEmbeddings(model="llama2")

# Criação de vetor vazio
vectorstore = Chroma(embedding_function=embeddings, persist_directory="chroma_db")

# Adiciona os chunks
for chunk in tqdm(chunks, desc="Gerando embeddings"):
    vectorstore.add_documents([chunk])

/tmp/ipykernel_8967/394355359.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings, persist_directory="chroma_db")
Gerando embeddings: 100%|███████████████████████████████████████████| 273/273 [49:57<00:00, 10.98s/it]


In [10]:
from langchain.llms import Ollama

llm = Ollama(model="llama2")

/tmp/ipykernel_8967/3081704472.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [22]:
# Criar pipeline RAG
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=vectorstore.as_retriever())

In [23]:
# Consultas
pergunta = "Quais são as principais técnicas de teste? Responda em português brasileiro"

In [24]:
# Resposta sem RAG
print(f'{pergunta}\n{llm(pergunta)}')

Quais são as principais técnicas de teste? Responda em português brasileiro

 Existem várias técnicas de teste que podem ser utilizadas para avaliar a qualidade de um software. Aqui estão algumas das principais técnicas de teste:

1. Testes unitários: são realizados individualmente para cada componente do software, com o objetivo de verificar se ele funciona corretamente sozinho.
2. Testes de integração: são realizados após os testes unitários, para verificar se os componentes trabalham juntos corretamente.
3. Testes de sistemas: são realizados para verificar se o software funciona corretamente no sistema de hospedagem e nos ambientes de produção.
4. Testes de usabilidade: são realizados para avaliar se o software é fácil de usar e oferece uma boa experiência ao usuário.
5. Testes de aceitação: são realizados com o objetivo de verificar se o software atende às necessidades dos clientes ou usuários.
6. Testes de vulnerabilidade: são realizados para detectar possíveis vulnerabilidades de

In [25]:
# Resposta com o pipeline RAG
print(f'{pergunta}\n{qa_chain.run(pergunta)}')

Quais são as principais técnicas de teste? Responda em português brasileiro
As principais técnicas de teste são:

1. Teste antecipado: consiste em realizar testes durante a fase de desenvolvimento, para identificar defeitos e melhorar a qualidade do produto de forma anticipada.
2. Teste de cobertura: consiste em testar todo o código e todas as funcionalidades do produto, para garantir que todos os requisitos sejam atendidos.
3. Teste de critério: consiste em testar o produto com base em critérios específicos, como performance, segurança, etc.
4. Teste de usabilidade: consiste em testar a facilidade de uso do produto e a experiência do usuário, para garantir que ele seja intuitivo e fácil de usar.
5. Teste de automação: consiste em utilizar tecnologias de automação de testes para automatizar o teste de funcionamentos e outras tarefas de teste.
6. Teste de manual: consiste em realizar testes manualmente, sem a ajuda de tecnologias de automação.
7. Teste de combinação: consiste em combina

In [26]:
# Criação de prompt personalizado
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
Você é um especialista em testes de software e foi delegado a tarefa de responder perguntas baseando-se no Syllabus do ISTOQB.

Na construção de suas resposta, utilize **apenas o conteúdo disponível nos documentos a seguir**.

Seja conciso, incluindo em sua resposta apenas os bullet points e uma referência à seção em que a informação se encontra.

{context}

Pergunta: {question}
Resposta:
"""

prompt = PromptTemplate(input_variables=["context", "question"],
                        template=template,)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt":prompt},
)

resposta = qa_chain.invoke({"query":pergunta})
print(f'{resposta["query"]}\n{resposta["result"]}')

Quais são as principais técnicas de teste? Responda em português brasileiro
para responder à sua pergunta, irei basear-me nos points disponíveis no syllabus do ISTQB:

1.5.3 Independência dos Testes: Todas as atividades de desenvolvimento devem estar sujeitas ao controle de qualidade.

* Diferentes níveis de teste têm objetivos de teste específicos e diferentes, o que permite que os testes sejam adequadamente abrangentes, evitando redundância.
* A análise e a modelagem do teste para um determinado nível de teste começam durante a fase de desenvolvimento correspondente do SDLC, para que o teste possa aderir ao princípio do teste antecipado.

2.2.1 Testes diferentes: Os testes têm objetivos específicos e diferentes, dependendo do nível de teste e da etapa do desenvolvimento.

* A abordagem de equipe completa pode ser adequada em situações, como críticas para a segurança, onde é necessário um alto nível de independência dos testes.

2.1.5 Shift-left: A estratégia shift-left consiste em en

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/aiel/Documentos/faculdade/ia/RAG/env/lib/python3.11/site-packages/colabxterm/__main__.py", line 14, in <module>
    term.open()
  File "/home/aiel/Documentos/faculdade/ia/RAG/env/lib/python3.11/site-packages/colabxterm/xterm.py", line 83, in open
    tornado.ioloop.IOLoop.current().start()
  File "/home/aiel/Documentos/faculdade/ia/RAG/env/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
    self._run_once()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1884, in _run_once
    event_list = self._selector.select(timeout)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/selectors.py", line 468, in select
    fd_event_list = self._selector.poll(timeout, max_ev

# RAG
O RAG é uma técnica de consulta de LLMs a fim de adquirir resultados mais precisos.
Com tal objetivo em mente, um prompt personalizado é construído. Esse prompt é feito de uma forma que inclui um contexto ao qual o modelo deve se basear na geração de sua resposta, por isso o nome: Retrival-Augmented Generation.

Para repassar um contexto para um modelo já treinado, é preciso fazer um pré-processamento dele, transformando o contexto em uma linguagem que o modelo possa entender. Mas antes, o contexto é dividido em pedaços menores, **chunks**, para garantir uma maior precisão da recuperação, reduzindo as chances de uma informação ser obscurecida em um documento muito grande. Somente então é feito o **embedding**, onde os chunks do contexto passado são transformados em vetores de múltiplas dimensões. Estes vetores extraem o sentido geral de cada chunk e são eles que são repassados ao modelo para uma consulta.

A utilização do RAG garante um maior alinhamento do modelo com o contexto passado. Esse alinhamento pode ser compreendido como uma maior precisão do modelo, visto que o contexto passado tende a conter todas as informações que o modelo precisará, em tal aplicação. Por esse motivo, RAGs fazem-se úteis em contextos onde é esperado do modelo um certo comprometimento com as informações a serem repassadas, como no caso de chatbots.

A diferença entre os resultados obtidos nas consultas acima são nítidos. Os modelos são capazes de gerar respostas abstratas para qualquer pergunta feita, mas, com a utilização do RAG, a resposta gerada se aproxima do documento passado, não precisando gerar uma resposta vaga, a mesma pergunta passa a ter um diferente sentido devido ao contexto. 